In [441]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [442]:
model_name = "lstm"

time_list = [0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15]
arr_rate_list = [30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90]
power_list = np.arange(10, 51, 1)

if model_name == 'bert':
    time_list = [1, 1.2, 1.4, 1.6, 1.8, 2, 2.2, 2.4]
    arr_rate_list = [1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
    power_list = np.arange(10, 61, 1)


In [443]:
len(time_list)*len(arr_rate_list)*len(power_list)

5863

In [444]:
sampled_pwds = pd.read_csv("tl/e24_random_50_pwmds.csv")

In [445]:
sampled_pwds = list(sampled_pwds['powermode'].values)

In [446]:
all_data = pd.read_csv(f"{model_name}_infer_data_final.csv")

In [447]:
all_data['powermode'] = all_data['cores'].astype(str) + "_" + all_data['cpu'].astype(str) + "_" + all_data['gpu'].astype(str) + "_" + all_data['mem'].astype(str)

In [448]:
# filter out the sampled pwds
all_data = all_data[all_data['powermode'].isin(sampled_pwds)]

In [449]:
all_data.reset_index(drop=True, inplace=True)

In [450]:
all_data

,cores,cpu,gpu,mem,observed_time,bs,observed_power,powermode
0,4,422400,114750000,3199000000,12.006416,1.0,14.810,4_422400_114750000_3199000000
1,12,422400,114750000,665600000,11.383920,1.0,13.590,12_422400_114750000_665600000
2,8,422400,726750000,2133000000,7.616560,1.0,18.015,8_422400_726750000_2133000000
3,12,422400,930750000,3199000000,8.803392,1.0,17.119,12_422400_930750000_3199000000
4,4,422400,1300500000,3199000000,6.401792,1.0,19.231,4_422400_1300500000_3199000000
...,...,...,...,...,...,...,...,...
245,8,2201600,114750000,665600000,8.443056,64.0,15.215,8_2201600_114750000_665600000
246,12,2201600,318750000,2133000000,2.114128,64.0,29.497,12_2201600_318750000_2133000000
247,12,2201600,522750000,3199000000,2.585136,64.0,23.163,12_2201600_522750000_3199000000
248,12,2201600,726750000,3199000000,2.255264,64.0,24.771,12_2201600_726750000_3199000000


In [451]:
nn_pareto_df = pd.read_csv(f"/home/saisamarth/exp/tl/e24_merged_observed_predicted_{model_name}_NN_infer_sampled_250.csv")


In [452]:
nn_pareto_df = nn_pareto_df[nn_pareto_df['powermode'].isin(sampled_pwds)]

In [453]:
nn_pareto_df.reset_index(inplace=True, drop=True)

In [454]:
nn_pareto_df[nn_pareto_df['powermode']=='8_1958400_318750000_2133000000']

,powermode,bs,observed_time,predicted_time,observed_power,predicted_power
36,8_1958400_318750000_2133000000,1,2.245744,2.300541,26.875,24.955650
86,8_1958400_318750000_2133000000,4,3.487168,3.233445,18.733,21.038034
136,8_1958400_318750000_2133000000,16,3.490128,3.396637,18.834,18.390312
186,8_1958400_318750000_2133000000,32,3.478528,3.389044,18.838,18.398588
236,8_1958400_318750000_2133000000,64,2.233296,2.496119,27.284,24.312056


In [455]:
nn_pareto_df[nn_pareto_df['observed_time'] < 200]

,powermode,bs,observed_time,predicted_time,observed_power,predicted_power
0,4_422400_114750000_3199000000,1,12.006416,10.642159,14.810,14.811188
1,12_422400_114750000_665600000,1,11.383920,10.603813,13.590,14.189640
2,8_422400_726750000_2133000000,1,7.616560,6.995014,18.015,17.397955
3,12_422400_930750000_3199000000,1,8.803392,8.269593,17.119,17.142730
4,4_422400_1300500000_3199000000,1,6.401792,6.224821,19.231,18.881124
...,...,...,...,...,...,...
245,8_2201600_114750000_665600000,64,8.443056,8.156893,15.215,15.195837
246,12_2201600_318750000_2133000000,64,2.114128,2.166153,29.497,28.434397
247,12_2201600_522750000_3199000000,64,2.585136,2.570416,23.163,22.928010
248,12_2201600_726750000_3199000000,64,2.255264,2.487618,24.771,24.094213


In [456]:
power_list = np.arange(10, 51, 1)

In [457]:
def find_optimal_pm(df, pmax, tmax, a):
    #filter rows where interleaved_power<=pmax, sort in ascending order of interleaved_time
    all_data = df.copy()
    df = df[df['observed_power'] <= pmax]
    df['time_scaled'] = df['observed_time']/1000.0
    df['time_cond1'] = (df['bs'].astype(int))/a + df['time_scaled']
    df['time_cond2'] = df['time_scaled']*a

    df = df[(df['time_cond1'] <= tmax)]
    df = df[(df['time_cond2'] <= df['bs'])]
    
    df = df.sort_values(by='time_scaled', ascending=True)

    if df.empty:
        temp_data = all_data[all_data['observed_power'] > pmax]
        if temp_data.empty:
            all_data = all_data.sort_values(by='observed_time', ascending=True)
            return all_data.iloc[0]['powermode'], all_data.iloc[0]['bs'], all_data.iloc[0]['observed_time'], all_data.iloc[0]['observed_power'], 0
        else:
            temp_data = temp_data.sort_values(by='observed_time', ascending=True)
            return temp_data.iloc[0]['powermode'], temp_data.iloc[0]['bs'], temp_data.iloc[0]['observed_time'], temp_data.iloc[0]['observed_power'], 0
            
    else:
        return df.iloc[0]['powermode'], df.iloc[0]['bs'], df.iloc[0]['observed_time'], df.iloc[0]['observed_power'], 1

In [458]:
def find_optimal_pm_nn(df, pmax, tmax, a):
    #filter rows where interleaved_power<=pmax, sort in ascending order of interleaved_time
    all_data = df.copy()
    df = df[df['predicted_power'] <= pmax]
    df['time_scaled'] = df['predicted_time']/1000.0
    df['time_cond1'] = (df['bs'].astype(int))/a + df['time_scaled']
    df['time_cond2'] = df['time_scaled']*a

    df = df[(df['time_cond1'] <= tmax)]
    df = df[(df['time_cond2'] <= df['bs'])]
    
    df = df.sort_values(by='time_scaled', ascending=True)
    if df.empty:
        temp_data = all_data[all_data['predicted_power'] > pmax]
        if temp_data.empty:
            all_data = all_data.sort_values(by='predicted_time', ascending=True)
            return all_data.iloc[0]['powermode'], all_data.iloc[0]['bs'], all_data.iloc[0]['observed_time'], all_data.iloc[0]['observed_power'], 0
        else:
            temp_data = temp_data.sort_values(by='predicted_time', ascending=True)
            return temp_data.iloc[0]['powermode'], temp_data.iloc[0]['bs'], temp_data.iloc[0]['observed_time'], temp_data.iloc[0]['observed_power'], 0
    #return top1 powermode, time and power
    return df.iloc[0]['powermode'], df.iloc[0]['bs'], df.iloc[0]['observed_time'], df.iloc[0]['observed_power'], 1

In [459]:
random_result_df = pd.DataFrame(columns=['power_budget', 'powermode', 'bs', 'optim_time', 'optim_power','time_budget', 'arr_rate','soln'])

for arr_rate in arr_rate_list:
    for time_budget in time_list:
        for power_budget in power_list:
            try:
                powermode, bs, algo_time, algo_power, soln = find_optimal_pm(all_data, power_budget, time_budget, arr_rate)
                random_result_df.loc[len(random_result_df)] = [power_budget, powermode, bs, algo_time, algo_power, time_budget, arr_rate, soln]
            except:
                print("No Power Mode Found")
                random_result_df.loc[len(random_result_df)] = [power_budget, 0, 0, 0, 0, time_budget, arr_rate, 0]

In [460]:
result_df_nn = pd.DataFrame(columns=['power_budget', 'powermode', 'bs', 'optim_time', 'optim_power', 'time_budget', 'arr_rate','soln'])

for arr_rate in arr_rate_list:
    for time_budget in time_list:
        for power_budget in power_list:
            try:
                powermode, bs, algo_time, algo_power, soln = find_optimal_pm_nn(nn_pareto_df, power_budget, time_budget, arr_rate)
                result_df_nn.loc[len(result_df_nn)] = [power_budget, powermode, bs, algo_time, algo_power, time_budget, arr_rate, soln]
            except:
                print("No Power Mode Found")
                result_df_nn.loc[len(result_df_nn)] = [power_budget, 0, 0, 0, 0, time_budget, arr_rate, 0]

In [461]:
# merge the two dataframes on power_budget with suffixes (random, nn)
merged_df = pd.merge(random_result_df, result_df_nn, on=['power_budget', 'time_budget', 'arr_rate'], suffixes=('_random', '_nn'))

In [462]:
merged_df.columns

Index(['power_budget', 'powermode_random', 'bs_random', 'optim_time_random',
       'optim_power_random', 'time_budget', 'arr_rate', 'soln_random',
       'powermode_nn', 'bs_nn', 'optim_time_nn', 'optim_power_nn', 'soln_nn'],
      dtype='object')

In [463]:
def validate_nn_solution(data):
    data['flag'] = True
    # check if optim_power_nn is less than power_budget, flag = False
    data.loc[data['optim_power_nn'] > (data['power_budget'] + 1), 'flag'] = False
    data['time_scaled'] = data['optim_time_nn']/1000.0
    data['time_cond1'] = (data['bs_nn'].astype(int))/data['arr_rate'] + data['time_scaled']
    data['time_cond2'] = data['time_scaled']*data['arr_rate']
    # check if time_cond1 is less than time_budget, flag = False
    data.loc[data['time_cond1'] > (data['time_budget'] + 0.01), 'flag'] = False
    # check if time_cond2 is less than bs, flag = False
    data.loc[data['time_cond2'] > data['bs_nn'], 'flag'] = False
    return data


In [464]:
merged_df = validate_nn_solution(merged_df)

In [465]:
merged_df

,power_budget,powermode_random,bs_random,optim_time_random,optim_power_random,time_budget,arr_rate,soln_random,powermode_nn,bs_nn,optim_time_nn,optim_power_nn,soln_nn,flag,time_scaled,time_cond1,time_cond2
0,10,4_2201600_930750000_2133000000,1.0,2.089504,28.384,0.05,30,0,4_1958400_522750000_2133000000,64,2.237760,26.882,0,False,0.002238,2.135571,0.067133
1,11,4_2201600_930750000_2133000000,1.0,2.089504,28.384,0.05,30,0,4_1958400_522750000_2133000000,64,2.237760,26.882,0,False,0.002238,2.135571,0.067133
2,12,4_2201600_930750000_2133000000,1.0,2.089504,28.384,0.05,30,0,4_1958400_522750000_2133000000,64,2.237760,26.882,0,False,0.002238,2.135571,0.067133
3,13,4_2201600_930750000_2133000000,1.0,2.089504,28.384,0.05,30,0,4_1958400_522750000_2133000000,64,2.237760,26.882,0,False,0.002238,2.135571,0.067133
4,14,12_422400_114750000_665600000,1.0,11.383920,13.590,0.05,30,1,4_1958400_522750000_2133000000,64,2.237760,26.882,0,False,0.002238,2.135571,0.067133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5858,46,4_2201600_930750000_2133000000,1.0,2.089504,28.384,0.15,90,1,12_2201600_318750000_2133000000,1,2.108832,29.095,1,True,0.002109,0.013220,0.189795
5859,47,4_2201600_930750000_2133000000,1.0,2.089504,28.384,0.15,90,1,12_2201600_318750000_2133000000,1,2.108832,29.095,1,True,0.002109,0.013220,0.189795
5860,48,4_2201600_930750000_2133000000,1.0,2.089504,28.384,0.15,90,1,12_2201600_318750000_2133000000,1,2.108832,29.095,1,True,0.002109,0.013220,0.189795
5861,49,4_2201600_930750000_2133000000,1.0,2.089504,28.384,0.15,90,1,12_2201600_318750000_2133000000,1,2.108832,29.095,1,True,0.002109,0.013220,0.189795


In [466]:
# find the number of times flag is false
merged_df[merged_df['flag']==False].shape

(899, 17)

In [467]:
# time = int(time_budget*1000)

In [468]:
merged_df.to_csv(f"{model_name}_random_vs_nn_infer_multi_al.csv", index=False)